In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import LineCollection

import json
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.integrate as integrate
from sklearn import linear_model

from collections import Counter

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('axes', **{'titlesize': '16', 'labelsize': '16'})
rc('legend', **{'fontsize': '16'})
rc('figure', **{'dpi' : 200})

In [ ]:
# Наблюдения
y = np.array([2,2,2,2,3,4,6,8,10,10,10,10,15,20,25,30,45,59,63,93,114,147,199,253,306,438,438,495,658,840,1036,1264,1534,1836,2337,2777,3548,4149,4731,5389,6343,7497,8672])
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1,1,1)
ax.plot(y)
T, N = len(y), 20000
init_rho, init_mu, init_beta = 0.01, 0.01, 0.00001
prior_rho_params = (1, 99)
prior_mu_params = (1, 99)
prior_beta_params = (1, 9999)

In [ ]:
class SIR_MCMC:
    
    def __init__(self, prior_rho_params, prior_mu_params, prior_beta_params):
        self.param_names = ['rho', 'mu', 'beta']
        self.params = { 'rho' : prior_rho_params, 'mu' : prior_mu_params, 'beta' : prior_beta_params }
        self.params_history = { i : [x] for i,x in self.params.items() }        
        self._sample_parameters(use_means=True)

    def _sample_parameters(self, use_means=False):
        if use_means:
            self.rho, self.mu, self.beta = [ self.params[x][0] / (self.params[x][0] + self.params[x][1]) for x in self.param_names ]
        else:
            self.rho, self.mu, self.beta = [ np.random.beta(self.params[x][0], self.params[x][1]) for x in self.param_names ]
        
        self.logbeta = np.log(self.beta)
        self.logmu = np.log(self.mu)
        self.log1mbeta = np.log(1 - self.beta)
        self.log1mmu = np.log(1 - self.mu)

    def _sample_initial_trajectories(self):
        print("\t\t...rho = %.6f\tmu = %.6f\tbeta = %.6f" % (self.rho, self.mu, self.beta))
        iIter = 0
        while True:
            iIter += 1
            if iIter % 100 == 0:
                self._sample_parameters()
                print("\t\t...rho = %.6f\tmu = %.6f\tbeta = %.6f" % (self.rho, self.mu, self.beta))
            self.xs = np.zeros(shape=(self.N, 2)) - 1
            self.xs[0:self.y[0], 0] = 0
            self.arr_S, self.arr_I, self.arr_R = [self.N-self.y[0]], [self.y[0]], [0]
            
            for t in range(1, self.T):
                # переходы из S в I
                next_S_to_I = np.random.binomial(self.arr_S[-1], -np.expm1(self.arr_I[-1]*np.log(1-self.beta)) )

                # переходы из I в R
                next_I_to_R = np.random.binomial(self.arr_I[-1], self.mu)
                
                # проверяем, что вероятность такой последовательности не ноль
                if self.arr_I[-1] + next_S_to_I - next_I_to_R < self.y[t]:
                    break
                
                # обновляем траектории
                self.xs[self.arr_I[-1] + self.arr_R[-1]:self.arr_I[-1] + self.arr_R[-1] + next_S_to_I, 0] = t
                self.xs[self.arr_R[-1]:self.arr_R[-1] + next_I_to_R, 1] = t

                # обновляем массивы
                self.arr_S.append(self.arr_S[-1] - next_S_to_I)
                self.arr_I.append(self.arr_I[-1] + next_S_to_I - next_I_to_R)
                self.arr_R.append(self.arr_R[-1] + next_I_to_R)

            if len(self.arr_S) >= self.T:
                break

    def _update_state_arrays(self):
        self.arr_I = np.array([ np.sum(np.logical_and(np.logical_and(self.xs[:,0] >= 0, self.xs[:,0] <= t), np.logical_or(self.xs[:,1] < 0, self.xs[:,1] >= t))) for t in range(self.T)])
        self.arr_S = np.array([ np.sum(np.logical_or(self.xs[:,0] < 0, self.xs[:,0] > t)) for t in range(self.T)])
        self.arr_R = self.N - self.arr_I - self.arr_S
                
    def _compute_Q_matrices(self, ind_x):
        cur_x = self.xs[ind_x]
        
        ## считаем всё в логарифмах
        qs = np.zeros(shape=(self.T, 3, 3)) - np.inf
        if cur_x[0] == 0:
            qs[0,0,1] = 0.0
        else:
            qs[0,0,0] = 0.0

        if cur_x[1] == -1:
            if cur_x[0] > -1:
                arr_I_without_current = self.arr_I - np.array( np.arange(self.T) >= cur_x[0], dtype=int)
            else:
                arr_I_without_current = self.arr_I
        else:
            arr_I_without_current = self.arr_I - np.array( np.logical_and(np.arange(self.T) >= cur_x[0], np.arange(self.T) < cur_x[1]), dtype=int)
        
        for t in range(1, self.T):
            logprob_y_without_current = sp.stats.binom.logpmf(self.y[t], arr_I_without_current[t], self.rho)
            logprob_y_with_current = sp.stats.binom.logpmf(self.y[t], arr_I_without_current[t]+1, self.rho)
            qs[t] = -np.inf
            if qs[t-1, 0, 0] > -np.inf:
                logprob_stay_healthy = arr_I_without_current[t-1] * self.log1mbeta
                qs[t, 0, 0] = (qs[t-1, 0, 0]) + logprob_stay_healthy + logprob_y_without_current
                qs[t, 0, 1] = (qs[t-1, 0, 0]) + np.log(-np.expm1(logprob_stay_healthy)) + logprob_y_with_current
            prob_prev_I_state, prob_prev_R_state = np.logaddexp.reduce(qs[t-1, :, 1]), np.logaddexp.reduce(qs[t-1, :, 2])
            if prob_prev_I_state > -np.inf:
                qs[t, 1, 1] = (prob_prev_I_state) + self.log1mmu + logprob_y_with_current
                qs[t, 1, 2] = (prob_prev_I_state) + self.logmu + logprob_y_without_current
            if prob_prev_R_state > -np.inf:
                qs[t, 2, 2] = (prob_prev_R_state) + logprob_y_without_current
            qs[t] = qs[t] - np.logaddexp.reduce(qs[t], axis=(0,1))

        ## экспоненту берём только на выходе
        return np.exp(qs)
          
    def _sample_new_x(self, qs):        
#         print(qs)
        new_x = np.zeros(2)
        cur_state = np.random.choice(3, p=np.sum(qs[self.T-1], axis=1))
        new_x[cur_state:] = -1
        for t in range(T-2, -1, -1):
            new_state = np.random.choice(3, p=qs[t+1,:,cur_state] / np.sum(qs[t+1,:,cur_state]))
            if new_state < cur_state:
                new_x[new_state] = t+1
                cur_state = new_state
            if new_state == 0:
                break
        return new_x

    def _update_state_arrays(self):
        self.arr_S = np.array([ np.sum(np.logical_or(self.xs[:,0] < 0, self.xs[:,0] > t)) for t in range(self.T)])
        self.arr_R = np.array([ np.sum(np.logical_and(self.xs[:,1] >= 0, self.xs[:,1] <= t)) for t in range(self.T)])
        self.arr_I = self.N - self.arr_S - self.arr_R

    def _metropolis_hastings_loop(self, iters):
        for _ in range(iters):
            x_index = np.random.randint(self.N)
            qs = self._compute_Q_matrices(x_index)
            new_x = self._sample_new_x(qs)
            self.xs[x_index] = new_x
            self._update_state_arrays()

    ## обновляем параметры модели
    def _update_model_parameters(self):
        # сколько человек заболело и не заболело в каждый момент
        indices_infections =  np.array( self.xs[self.y[0]:,0]-1, dtype=int )
        counter_stayhealthy = Counter( [ i for x in indices_infections for i in range(0, x)] )
        counter_infections = Counter( indices_infections )
        
        # сколько ожидаемых инфицирований на каждого инфицированного
        expected_infections = np.divide( self.beta*self.arr_I, -np.expm1( self.arr_I * self.log1mbeta ) )
        expected_infections[ np.where(self.arr_I == 1) ] = 1
        expected_infections[ np.where(self.arr_I == 0) ] = 0

        # суммарные ожидаемые числа событий инфицирований и не-инфицирований
        expected_num_I_infections = np.sum([ x * expected_infections[i] for i,x in counter_infections.items()])
        expected_num_I_notinfections = np.sum([x * (self.arr_I[i] - expected_infections[i]) for i,x in counter_infections.items()])
        expected_num_S_notinfections = np.sum([ x * self.arr_I[i] for i,x in counter_stayhealthy.items() ])
                
        for x in self.param_names:
            self.params_history[x].append(self.params[x])
        current_param_updates = {
            'rho': ( np.sum(self.y), np.sum(self.arr_I) - np.sum(self.y) ),
            'mu': ( np.sum(np.diff(self.arr_R)), np.sum(self.arr_I[:-1]) - np.sum(np.diff(self.arr_R)) ),
            'beta': ( expected_num_I_infections, expected_num_I_notinfections + expected_num_S_notinfections )
        }
        for x in self.param_names:
            self.params[x] = (self.params_history[x][0][0] + current_param_updates[x][0], self.params_history[x][0][1] + current_param_updates[x][1])

            
    ### the main loop
    def fit(self, y, N, iters=3, mh_iters=500, burn_in=500, verbose=True):
        self.y, self.N, self.T = y, N, y.shape[0]
        self._sample_initial_trajectories()
        
        if verbose:
            print("Initial values:\trho = %.6f\tmu = %.6f\tbeta = %.6f" % (self.rho, self.mu, self.beta))

        self._metropolis_hastings_loop(burn_in)
            
        for iIter in range(iters):
            self._metropolis_hastings_loop(mh_iters)
            self._update_model_parameters()
            if verbose:
                print('New params: %s' % "\t".join(["%s: (%.2f,%.2f)" % (x, self.params[x][0], self.params[x][1]) for x in self.param_names]))
            self._sample_parameters(use_means=True)
            if verbose:
                print("Iteration %2d:\trho = %.6f\tmu = %.6f\tbeta = %.6f" % (iIter, self.rho, self.mu, self.beta))


In [ ]:
def sample_population(N, T, true_rho, true_beta, true_mu):
    true_log1mbeta = np.log(1 - true_beta)

    cur_states = np.zeros(N)
    cur_states[:1] = 1
    cur_I, test_y, true_statistics = 1, [1], [[N-1, 1, 0]]

    for t in range(T):    
        logprob_stay_healthy = cur_I * true_log1mbeta
        for i in range(N):
            if cur_states[i] == 0 and np.random.rand() < -np.expm1(logprob_stay_healthy):
                cur_states[i] = 1
            elif cur_states[i] == 1 and np.random.rand() < true_mu:
                cur_states[i] = 2

        cur_I = np.sum(cur_states == 1)
        test_y.append( np.random.binomial( cur_I, true_rho ) )
        true_statistics.append([np.sum(cur_states == 0), np.sum(cur_states == 1), np.sum(cur_states == 2)])

    return test_y, np.array(true_statistics).T

In [ ]:
N, T, true_rho, true_beta, true_mu = 100, 20, 0.1, 0.05, 0.2
test_y, true_statistics = sample_population(N, T, true_rho, true_beta, true_mu)
print(np.array(test_y))
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1,1,1)
ax.plot(true_statistics.T)
ax.legend(["S", "I", "R"])
ax.set_xlim((0, 20))
# fig.savefig('covid1.pdf', bbox_inches='tight')
# plt.savefig("covid1.png", bbox_inches='tight')

In [ ]:
print("True values: rho=%.4f\tmu=%.4f\tbeta=%.4f" % (true_rho, true_mu, true_beta))
print("Y = %s" % test_y)
model = SIR_MCMC((1, 1), (1, 1), (1, 10))
model.fit(np.array(test_y), N, iters=3000, mh_iters=int(N/10), burn_in=2*N)

In [ ]:
def beta_mean(x):
    return x[0] / (x[0] + x[1])

true_params ={'rho' : true_rho, 'mu': true_mu, 'beta' : true_beta}

def plot_model_params_history(ax, model, true_params=None):
    ## нарисуем, как менялись параметры модели со временем
    for i, name in enumerate(model.param_names):
        ax.plot([beta_mean(x) for x in model.params_history[name]], color=palette[i])
        if true_params is not None:
            ax.hlines(true_params[name], xmin=0, xmax=len(model.params_history[name]), linestyle='dashed', color=palette[i])
    ax.legend(model.param_names)

In [ ]:
print(true_params)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1,1,1)
plot_model_params_history(ax, model, true_params)
ax.set_ylim((0, 0.35))
ax.set_xlim((0, 1500))
# plt.savefig('covid2.png', bbox_inches='tight')

for i, name in enumerate(model.param_names):
    print("%s: global=%.05f\tlast100=%.05f" % (name, beta_mean( (np.sum([x[0] for x in model.params_history[name]]), np.sum([x[1] for x in model.params_history[name]]) )), beta_mean( (np.sum([x[0] for x in model.params_history[name][-100:]]), np.sum([x[1] for x in model.params_history[name][-100:]]) ))   ))

In [ ]:
def plot_model_and_true_arrays(ax, true_stats, model):
    for i,line in enumerate(true_stats):
        ax.plot(line, color=palette[i])
    for i,line in enumerate([model.arr_S, model.arr_I, model.arr_R]):
        ax.plot(line, color=palette[i], linestyle='dashed')
    ax.legend(["S", "I", "R", "S model", "I model", "R model"])

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1,1,1)
plot_model_and_true_arrays(ax, true_statistics, model)
ax.set_xlim((0, 20))
# fig.savefig('covid3.png', bbox_inches='tight')

mean_params = {
   name : beta_mean( (np.sum([x[0] for x in model.params_history[name][-100:]]), np.sum([x[1] for x in model.params_history[name][-100:]]) ))
    for name in model.param_names }

print("Observed data: %s" % test_y)
true_sample, model_sample, mean_sample = [], [], []
for _ in range(500):
    model_sample.append(sample_population(N, T, model.rho, model.beta, model.mu)[0])
    true_sample.append(sample_population(N, T, true_rho, true_beta, true_mu)[0])
    mean_sample.append(sample_population(N, T, mean_params['rho'], mean_params['beta'], mean_params['mu'])[0])
    
print("Average sample from true parameters (dev=%.4f):\n%s" % (np.mean((np.array(true_sample)-test_y)**2), np.mean(np.array(true_sample), axis=0)))
print("Average sample from model parameters (dev=%.4f):\n%s" % (np.mean((np.array(model_sample)-test_y)**2), np.mean(np.array(model_sample), axis=0)))
print("Average sample from mean parameters (dev=%.4f):\n%s" % (np.mean((np.array(mean_sample)-test_y)**2), np.mean(np.array(mean_sample), axis=0)))